# Coordinate Descent to Cluster Cells

We want to perform coordinate descent to cluster cells. We will initialize a clustering using a common method, such as spectral clustering. Then, we will use that clustering to fit maximum likelihood embryo centers. Next, we will set the cluster of each cell according to maximum likelihood. This will be repeated until convergence.

In [1]:
import pandas as pd
import anndata
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from functools import partial
from multiprocessing import Pool
from sklearn.metrics import adjusted_rand_score
from itertools import product

## Load Data

In [2]:
NUM_EMBRYOS = 3

CALLS_SAVE_PATH = 'data/calls_2022_07_28_umi_filtered.h5ad'
READS_SAVE_PATH = 'data/reads_2022_07_28_umi_filtered.h5ad'

In [3]:
variants_joined = anndata.read_h5ad(CALLS_SAVE_PATH)
reads_joined = anndata.read_h5ad(READS_SAVE_PATH)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [8]:
inds = variants_joined.var.index

chrom = []
pos = []
transition = []

for ind in inds:
    split_ind = ind.split(':')
    chrom.append(split_ind[0])
    pos.append(split_ind[1])
    transition.append(split_ind[2])

In [10]:
counts = {position:0 for position in pos}

for position in pos:
    counts[position] += 1

In [18]:
np.histogram(np.array(list(counts.values())))

(array([27013,   146,    33,     4,     0,     4,     1,     1,     0,
            1]),
 array([1. , 1.8, 2.6, 3.4, 4.2, 5. , 5.8, 6.6, 7.4, 8.2, 9. ]))

## Run Descent

In [5]:
# Computes the log probability of each value of
# variant j in cluster i
def compute_best_log_prob_pi(j, i=None):
    # Get the calls for the variant for each cell in the cluster
    cell_calls = C[z == i, j]
    
    probs = []
    
    for pi_call in range(4):
        prob = 0
        
        for cell_call in cell_calls:
            if T[pi_call, cell_call] == 0:
                prob = -np.inf
                break
            
            prob += np.log(T[pi_call, cell_call])
            
        probs.append(prob)
        
    return np.argmax(probs)

# Sets pi by computing choosing the best log probability variant
# call for each cluster
def set_pi():
    pi = np.zeros((NUM_EMBRYOS, C.shape[1]), dtype=np.int8)
    
    for i in range(NUM_EMBRYOS):
        emb_prob_fun = partial(compute_best_log_prob_pi, i=i)
        
        p = Pool(NUM_THREADS)
        calls = list(tqdm(p.imap(emb_prob_fun, range(C.shape[1])), total=C.shape[1]))
        p.close()
                
        pi[i, :] = calls
            
    return pi

# Computes the log probabilities accross the possible clusters for
# C[i]
def compute_best_log_prob_C(i):
    probs = []
    
    for l in range(NUM_EMBRYOS):
        prob = 0
        
        for j in range(C.shape[1]):
            if T[pi[l, j], C[i, j]] == 0:
                prob = -np.inf
                break
                
            prob += np.log(T[pi[l, j], C[i, j]])
            
        probs.append(prob)
        
    return np.argmax(probs)


def get_row_counts(i):
    # Counts the number of transitions between what we think the actual embryo
    # value is (pi) and what we see in our clustered cell (C_i)
    count = np.zeros((4, 4))
    
    # Count the number of k1 -> k2 transitions
    for j in range(C.shape[1]):
        l = z[i]
        count[pi[l][j]][C[i][j]] += 1

    return count

def set_T():
    # Compute the most likely values for T by counting all the transitions    
    p = Pool(NUM_THREADS)
    counts = list(tqdm(p.imap(get_row_counts, range(C.shape[0])), total=C.shape[0]))
    p.close()
    
    T_new = np.zeros((4, 4))
    
    # Add together all cell counts
    for count in counts:
        T_new += count
        
    # Row normalize to make T a probability
    for i in range(4):
        row_sum = np.sum(T[i, :])
        
        if row_sum > 0:
            T[i, :] = T[i, :]/row_sum     
        
    return T_new

def set_z():    
    p = Pool(NUM_THREADS)
    z = np.array(list(tqdm(p.imap(compute_best_log_prob_C, range(C.shape[0])), total=C.shape[0])))
    p.close()
                            
    return z

In [11]:
NUM_THREADS = 90

def set_gt(epsilon):
    global C, z, T
    
    # Our ground truth data
    C = variants_joined.X
    #z = np.random.randint(0, NUM_EMBRYOS, size=C.shape[0])
    z = np.array(variants_joined.obs.spectral_clusters)

    # ~0.5 best
    T = np.array([[1.        , 0.        , 0.        , 0.        ],
           [0.74778739, 0.25221261, 0.        , 0.        ],
           [0.84580846, 0.03814512, 0.10065051, 0.01539591],
           [0.74778739, 0.        , 0.        , 0.25221261]])

    T = T + epsilon
    T = T/T.sum(axis=1)

#0.935
# T = np.array([[0.8        , 0.0666        , 0.0666        , 0.0666        ],
#        [0.7, 0.2, 0.05       , 0.05        ],
#        [0.84580846, 0.070081273, 0.132586663, 0.047332063],
#        [0.7, 0.05    , 0.05        , 0.2]])
T = np.random.rand(4, 4)

for i in range(T.shape[0]):
    T[i, :] = T[i, :]/np.sum(T[i, :])

# T = np.array([[0.6        , 0.1333      , 0.1333        , 0.1333       ],
#        [0.74778739, 0.25221261, 0.        , 0.        ],
#            [0.84580846, 0.03814512, 0.10065051, 0.01539591],
#            [0.74778739, 0.        , 0.        , 0.25221261]])

C = variants_joined.X
z = np.array(variants_joined.obs.spectral_clusters)

# ~0.5 best
# T = np.array([[0.8        , 0.066        , 0.066        , 0.066        ],
#        [0.65, 0.25, 0.05     , 0.05       ],
#        [0.7, 0.075, 0.15, 0.075],
#        [0.65, 0.05        , 0.05      , 0.252]])
# T = T + 0.5
# T = T/T.sum(axis=1)

In [12]:
def find_opt():
    global pi, z
    
    score_prev = -1

    while 1:
        print('Setting pi')
        pi = set_pi()

        print('Setting z')
        z = set_z()
        
#         print('Setting T')
#         T = set_T()

        score = adjusted_rand_score(variants_joined.obs.embryo, z)
        print('ARI is: ', score)

        if score-score_prev == 0:
            break

        score_prev = score
        
    return score

In [13]:
score = find_opt()

Setting pi


  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

Setting z


...

Setting z


  0%|          | 0/16480 [00:00<?, ?it/s]

Process ForkPoolWorker-8425:
Process ForkPoolWorker-8387:
Process ForkPoolWorker-8432:
Process ForkPoolWorker-8441:
Process ForkPoolWorker-8379:
Process ForkPoolWorker-8377:
Process ForkPoolWorker-8455:
Process ForkPoolWorker-8434:
Process ForkPoolWorker-8444:
Process ForkPoolWorker-8453:
Process ForkPoolWorker-8375:
Process ForkPoolWorker-8384:
Process ForkPoolWorker-8412:
Process ForkPoolWorker-8371:
Process ForkPoolWorker-8390:
Process ForkPoolWorker-8436:
Process ForkPoolWorker-8460:
Process ForkPoolWorker-8392:
Process ForkPoolWorker-8443:
Process ForkPoolWorker-8393:
Process ForkPoolWorker-8395:
Process ForkPoolWorker-8431:
Process ForkPoolWorker-8404:
Process ForkPoolWorker-8454:
Process ForkPoolWorker-8374:
Process ForkPoolWorker-8433:
Process ForkPoolWorker-8405:
Process ForkPoolWorker-8381:
Process ForkPoolWorker-8417:
Process ForkPoolWorker-8411:
Process ForkPoolWorker-8378:
Process ForkPoolWorker-8421:
Process ForkPoolWorker-8397:
Process ForkPoolWorker-8458:
Process ForkPo

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


In [7]:
# Do a parameter sweep on epsilon
eps_range = np.linspace(0, 0.5, 20)
scores = []

for eps in eps_range:
    print(f'Doing epsilon = {eps}')
    set_gt(eps)
    score = find_opt()
    scores.append(score)

Doing epsilon = 0.0
Setting pi


  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

Setting z


...

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

  0%|          | 0/27462 [00:00<?, ?it/s]

Process ForkPoolWorker-14960:
Process ForkPoolWorker-14975:
Process ForkPoolWorker-14983:
Process ForkPoolWorker-14982:
Process ForkPoolWorker-14989:
Process ForkPoolWorker-14998:
Process ForkPoolWorker-14973:
Process ForkPoolWorker-15003:
Process ForkPoolWorker-14959:
Process ForkPoolWorker-14948:
Process ForkPoolWorker-14987:
Process ForkPoolWorker-14971:
Process ForkPoolWorker-14944:
Process ForkPoolWorker-15027:
Process ForkPoolWorker-15006:
Process ForkPoolWorker-14984:
Process ForkPoolWorker-14991:
Process ForkPoolWorker-15005:
Process ForkPoolWorker-14958:
Process ForkPoolWorker-15012:
Process ForkPoolWorker-15013:
Process ForkPoolWorker-14985:
Process ForkPoolWorker-14966:
Process ForkPoolWorker-14974:
Process ForkPoolWorker-15010:
Process ForkPoolWorker-15004:
Process ForkPoolWorker-15011:
Process ForkPoolWorker-14950:
Process ForkPoolWorker-15000:
Process ForkPoolWorker-14957:
Process ForkPoolWorker-14969:
Process ForkPoolWorker-14956:
Process ForkPoolWorker-14955:
Process Fo

  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/multiprocessing/process.py", line